In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

In [2]:
from joblib import Parallel, delayed
from tqdm import tqdm

In [3]:
import sys
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
import BeadDataFile
from discharge_tools import *

In [4]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

sys.path.append('/home/analysis_user/New_trap_code/SensitivityFramework/')
from signal_model_utilities import *

In [5]:
%matplotlib widget

In [6]:
%reload_ext autoreload
%autoreload 2

In [25]:
391-375-7.6/2+1

13.2

In [7]:
## force template
dist = 13.2
z_sep = -10
y_offset = 4
test_shaking378 = force_vs_time(dist*1e-6,z_sep*1e-6,100e-6,3,"z",10e-6, yuk_or_grav="yuk",offset_y=y_offset*1e-6, 
                     alpha=1e8, bead_size=3.8e-6)
shaking378_template = np.array(test_shaking378[1])*1.85/1.55

Your choice of lambda is not existing
Taking 1.00e-05 for lambda
order
A bead size of 3.80e-06 is selected
3.8e-06
3.8e-06
A bead size of 3.80e-06 is selected
3.8e-06
posvec
A bead size of 3.80e-06 is selected
3.8e-06
A separation of 1.30e-05 is selected
A height of -1.00e-05 is selected


In [23]:
## likelihood calculator stuff
ll = likelihood_analyser.LikelihoodAnalyser()
gravity_freqs = np.array([36, 33, 39])
# gravity_freqs = 3*np.arange(6,15)
# gravity_freqs = np.array([36])
gravity_freqs = np.array([18, 12])

In [9]:
bdf_discharge = load_dir(dirname='/data/new_trap/20200320/Bead1/Discharge/Discharge_after_Mass_20200402/After_spinning_again_20200403/',
                         start_file=230, max_file=40)
bdf_z = bdf_discharge[5:35]

530  files in folder
40  files loaded


In [10]:
fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_X_m300k_250s_1hz.h5'
tf_x = BeadDataFile.BeadDataFile(fname)
fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Y_m300k_250s_1hz.h5'
tf_y = BeadDataFile.BeadDataFile(fname)
fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
tf_z = BeadDataFile.BeadDataFile(fname)
gfw.build_transfer_function([tf_x, tf_y, tf_z], base_freq=1, number_of_harmonics=90, scale_freq=71, plot=False)

No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data


### NoShaking data

In [11]:
## 1000 files of shaking data for nose floor
data_bdfs2 = load_dir(dirname='/data/new_trap/20200518/Bead2/Shaking/Shaking375/',
                         file_prefix='Shaking375_3', start_file=0, max_file=100)

1000  files in folder
100  files loaded


In [24]:
bandwidth = 2
m1_zscale = gfw.build_z_response(bdf_z, drive_freq=71, charges=6, bandwidth=bandwidth, decimate=10)
gravity_scales = np.array([gfw.scale_Z2])* np.interp(gravity_freqs, gfw.tf_freq, gfw.tf_ffts[2])

***************************************************
Z2-amplitude:  2.69e+02
reduced chi2:  288.98586212705186
***************************************************
Z2-amplitude:  2.60e+02
reduced chi2:  658.9340028864549
***************************************************
Z2-amplitude:  2.74e+02
reduced chi2:  486.49073911245125
***************************************************
Z2-amplitude:  2.78e+02
reduced chi2:  662.0476432309312
***************************************************
Z2-amplitude:  2.80e+02
reduced chi2:  581.29439966364
***************************************************
Z2-amplitude:  2.74e+02
reduced chi2:  610.1516298108376
***************************************************
Z2-amplitude:  2.70e+02
reduced chi2:  341.72862648819574
***************************************************
Z2-amplitude:  2.66e+02
reduced chi2:  319.23470715664763
***************************************************
Z2-amplitude:  2.66e+02
reduced chi2:  482.87048405227034
****************

In [25]:
gravity_freqs2 = gravity_freqs

In [26]:
noises = []
noises_std =[]
for freq_ in gravity_freqs2:
    tmp_list = []
    for bb in data_bdfs2:
        tmp_list.append(np.std(bb.response_at_freq2('z', drive_freq=freq_, bandwidth=bandwidth)))
    noises.append(np.mean(tmp_list))
    noises_std.append(np.std(tmp_list))
# noises = noises
noises_std = np.array(noises_std)

In [27]:
noises_std, gravity_freqs

(array([13.46749503, 22.14226255]), array([18, 12]))

In [183]:
fname = r'/data/new_trap/20200518/Bead2/Shaking/Shaking374/Shaking374_12_15.h5'
bb = BeadDataFile.BeadDataFile(fname=fname)
np.std(bb.cant_pos[1])*np.sqrt(2)*50*2

100.47219233313973

In [31]:
def func1(i):
# for i,bdf_ in enumerate(data_bdfs2):
    print(i,'  ******************')
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200403/NoShaking_1_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)

    fit_kwargs = {'A': 0, 'phi': 0.2, 'sigma':13.85,
              'error_A': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma': 10,
              'limit_phi': [0, 2 * np.pi],
              'limit_A': [-1000, 1000], 'limit_sigma': [0, 1000000],
              'print_level': 0, 'fix_A': True, 'fix_phi': True, 'fix_sigma': True}
    mm_tmp = ll.find_mle_multiHarmoincs(x=bb.z2, template=shaking378_template, scales=gravity_scales, 
                                          signal_freqs=gravity_freqs, bandwidth=2, noises=noises_std/noises_std[0], 
                                          decimate=10, **fit_kwargs)
#     print(mm_tmp.fval)
    return mm_tmp.fval

def func2(i):
    print(i,'  ******************')
    fname = r'/data/new_trap/20200518/Bead2/Shaking/Shaking375/Shaking375_7_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)

    ## force template
    dist = 13.2
    z_sep = -10
    y_offset = 0
    test_shaking378 = force_vs_time(dist*1e-6,z_sep*1e-6,100e-6,3,"z",100e-6, yuk_or_grav="yuk",offset_y=y_offset*1e-6, 
                     alpha=1e8, bead_size=3.8e-6)
    shaking378_template = np.array(test_shaking378[1])*1.85/1.55

    fit_kwargs = {'A': 0, 'phi': 0.2, 'sigma':10.7,
              'error_A': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma': 10,
              'limit_phi': [0, 2 * np.pi],
              'limit_A': [-1000, 1000], 'limit_sigma': [0, 1000000],
              'print_level': 0, 'fix_A': False, 'fix_phi': True, 'fix_sigma': False}
    mm_tmp = ll.find_mle_multiHarmoincs(x=bb.z2, template=shaking378_template, scales=gravity_scales, 
                                          signal_freqs=gravity_freqs, bandwidth=bandwidth, noises=noises_std,
                                          decimate=10, **fit_kwargs)

    return mm_tmp.values[0]
    
# if __name__ == '__main__':
#     pool = Pool(processes=20)                         # Create a multiprocessing Pool
# #     mm_fixed_A4 = np.array(pool.map(func1, range(0,10)))  # process data_inputs iterable with pool
#     mm_mle_A13 = np.array(pool.map(func2, range(0,1000)))
#     pool.close()

In [32]:
test_mle = np.array(Parallel(n_jobs=40)(delayed(func2)(i) for i in tqdm(range(1000))))

100%|██████████| 1000/1000 [00:03<00:00, 302.78it/s]


In [33]:
np.mean(test_mle), np.std(test_mle), np.std(test_mle)/np.sqrt(1000)

(1.9157515073777704, 4.3087502198245975, 0.1362546456339692)

In [19]:
_,ax = plt.subplots()
data = list(test_mle)#+list(mm_mle_A10)
avg = np.mean(data)
var = np.var(data)
# From that, we know the shape of the fitted Gaussian.
pdf_x = np.linspace(-25,25,100)
pdf_y = 1.0/np.sqrt(2*np.pi*var)*np.exp(-0.5*(pdf_x-avg)**2/var)

ax.hist(data, range=(-25,25), bins=25, density=True)
# ax.hist(mm_mle_A10, range=(-25,25), bins=30, density=True, alpha=0.4)
# Then we plot :
ax.set(xlim=(-25,25), xlabel=r'$\hat{\alpha} \,[10^{8}]$')
ax.plot(pdf_x,pdf_y,'k--')

print('alpha MLE: ', avg, 'std: ', np.std(data)/np.sqrt(1000))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

alpha MLE:  0.11135188961686084 std:  0.179509366522724
